In [1]:

import json
import os
import pandas as pd
import pprint
import tensorflow as tf
import time
import numpy as np
from tensorflow import keras
from keras.layers import *
import numpy as np 
from keras.preprocessing import image
import cv2
import matplotlib.pyplot as plt
from scipy.misc.pilutil import imread
import numpy as np
from glob import glob
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd

#matplotlib.use('agg')
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers import UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
import tensorflow as tf
import cv2

Using TensorFlow backend.


In [2]:
#Evaluation metrics
from keras import backend as K
def iou_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
  union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
  iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
  return iou

def dice_coef(y_true, y_pred, smooth=1):
  intersection = K.sum(y_true * y_pred, axis=[1,2,3])
  union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
  dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
  return dice

In [3]:
#final function 1 as told in the instructions
def final_fun_1(X_test):
  input_height, input_width = 128, 128
  output_height, output_width = 128, 128
  n_classes = 39
  pred = []
  dependencies = {'iou_coef': iou_coef,'dice_coef':dice_coef}
  model = load_model('canetres50.h5',custom_objects=dependencies)
  %matplotlib inline
  path = glob(X_test)

  h = 0
  for i in path:
  
    if h ==2:
      break
  
    img = cv2.imread(i)
    img = np.float32(cv2.resize(img, (input_width, input_height))) / 127.5 - 1
    img = np.expand_dims(img, axis = 0)

    seg = model.predict(np.float32(img))
    seg_ = np.argmax(seg, axis = 3)
    seg_ = cv2.resize(np.uint8(seg_[0]), (output_width, output_height))
   
  
    cv2.imwrite('label_'+str(h)+'.png',seg_)
    pred.append('label_'+str(h)+'.png')
  
    h+=1

  return pred


In [15]:
path = 'IDD_Canet/new_dataset/leftImg8bit/test/*_leftImg8bit.png'
pred = final_fun_1(path)

In [16]:
pred

['label_0.png', 'label_1.png']

In [4]:
def getImageArr(path, width, height):
  img = cv2.imread(path, 1)
  img = np.float32(cv2.resize(img, (width, height))) / 127.5 - 1
  return img

def getSegmentationArr(path, nClasses, width, height): 
  seg_labels = np.zeros((height, width, nClasses))
  img = cv2.imread(path, 1)
  img = cv2.resize(img, (width, height))
  #img = img[:, :, 0]

  for key in list(label_to_color_map.keys()):
        cur_color = np.array(label_to_color_map[key])[:3]
        img[np.where((img==cur_color).all(axis=2))] = int(label_to_id_map[key])
        
  img = img[:, :, 0]

  for c in range(nClasses):
    seg_labels[:, :, c] = (img == c).astype(int)

  return seg_labels

In [5]:
#Final function 2 as told in the instructions
def final_func_2(X_test,y_test):
  input_height, input_width = 128, 128
  output_height, output_width = 128, 128
  n_classes = 39
  pred = []
  dependencies = {'iou_coef': iou_coef,'dice_coef':dice_coef}
  model = load_model('canetres50.h5',custom_objects = dependencies)
  hist = model.evaluate(X_test,y_test,verbose = 0)
  
  return [hist[1]*100,hist[2]*100,hist[3]*100]


In [6]:
#paths to the images
image_paths_test = glob('IDD_Canet/new_dataset/leftImg8bit/test/*_leftImg8bit.png')
label_paths_test = [p.replace('new_dataset', 'labelled_Anue_dataset') for p in image_paths_test]

In [7]:
X_test, y_test = [], [] 

In [8]:
with open('label_to_color_map.txt') as f:
    label_to_color_map = json.load(f)
label_arr = np.arange(len(label_to_color_map.keys()))
color_arr = label_to_color_map.values()

id_to_label_map = {}
label_to_id_map = {}
color_to_id_map = {}
for i,class_id in enumerate(label_arr):
    label = list(label_to_color_map.keys())[i]
    color = list(label_to_color_map.values())[i]
    id_to_label_map[str(class_id)] = label
    label_to_id_map[label] = str(class_id)
    #color_to_id_map[color] = str(class_id)

In [9]:
from tqdm import tqdm
for i in tqdm(range(len(image_paths_test))):
  X_test.append(getImageArr(image_paths_test[i], 128,128))
  y_test.append(getSegmentationArr(label_paths_test[i], 39, 128, 128))

100%|██████████████████████████████████████████████████████████████████████████████| 1592/1592 [02:07<00:00, 12.50it/s]


In [10]:
X_test, y_test = np.array(X_test),np.array(y_test,dtype=np.uint8)
print(X_test.shape, y_test.shape)

(1592, 128, 128, 3) (1592, 128, 128, 39)


In [11]:
pred = final_func_2(X_test,y_test)

In [12]:
pred

[46.845704317092896, 62.64243125915527, 69.9500024318695]

In [13]:
#Evaluation metrics on test data
print("Mean IoU is "+ str(pred[0]))
print("Dice Coefficient is "+ str(pred[1]))
print("accuracy is "+ str(pred[2]))

Mean IoU is 46.845704317092896
Dice Coefficient is 62.64243125915527
accuracy is 69.9500024318695
